# Claude vs GPT

**Improvements**
- Pause the convo and continue without restarting.
- Add paramter to disable streaming in the backend as it is unecessary for our gradio function.
- Choose the topic ✅

In [ ]:
import sys
sys.path.append("..")

from src.gui import adversarial_chat
from src.test_chatbot import TestEris, TestSocrates
from src.chatbot import Socrates, Eris

from IPython.display import Markdown, display
import gradio as gr
import time
import random
import re
import numpy as np

from kokoro import KPipeline
from IPython.display import display, Audio
# import soundfile as sf
# import torch

d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

The system cannot find the path specified.


## Debate: Claude VS GPT

### Text-to-Speech

In [2]:
pipeline = KPipeline(lang_code='a', repo_id='hexgrad/Kokoro-82M')

d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


### GUI

In [ ]:
def clean_message(message):
    """
    Clean up AI-generated text for TTS:
    - Replace double newlines with a single newline.
    - Remove markdown or formatting characters (like **bold**, *italic*, etc.).
    - Strip markdown/code artifacts (e.g., ```code blocks```, >>> prompts).
    - Remove excess whitespace and trim.
    """
    if not isinstance(message, str):
        return ""

    # Replace triple backticks and markdown-style code blocks
    message = re.sub(r"```.*?```", "", message, flags=re.DOTALL)

    # Remove markdown formatting characters
    message = re.sub(r"\*\*|\*", "", message)        # bold/italic
    message = re.sub(r"_", "", message)              # underscore emphasis
    message = re.sub(r"#+", "", message)             # markdown headers like ###

    # Remove '>>>', often used in prompt examples
    message = message.replace(">>>", "")

    # Replace double newlines with single newlines
    message = re.sub(r"\n{2,}", "\n", message)

    # Remove stray symbols like •, →, etc.
    message = re.sub(r"[•→]", "", message)

    # Replace multiple spaces with a single space
    message = re.sub(r" {2,}", " ", message)

    # Remove leading/trailing whitespace
    return message.strip()


def estimate_audio_duration(audio_tuple):
    """
    Estimate audio duration from (sample_rate, audio_array) tuple.
    """
    if audio_tuple is None:
        return 0
    sample_rate, audio_array = audio_tuple
    return len(audio_array) / sample_rate


def speak_with_voice(text, is_socrates=True):
    """
    Generate speech with different voices for each character.
    Uses distinct voices for Socrates and Eris.
    """
    # Socrates gets a deeper, more contemplative voice
    # Eris gets a more playful, dynamic voice
    voice = "am_adam" if is_socrates else "af_heart"
    
    # Note: Replace this with your actual TTS pipeline
    generator = pipeline(text, voice=voice)
    for i, (_, _, audio_tensor) in enumerate(generator):
        audio_np = audio_tensor.numpy()
        return (24000, audio_np)
    
    # Placeholder return for demo purposes
    # sample_rate = 24000
    # duration = len(text) * 0.05  # Rough estimate: 0.05 seconds per character
    # audio_array = np.random.randn(int(sample_rate * duration)) * 0.1
    # return (sample_rate, audio_array)


def create_audio_history_item(message, is_socrates, audio_tuple):
    """
    Create a formatted audio history item with proper character names.
    """
    # Use full character names consistently
    speaker_name = "Socrates" if is_socrates else "Eris"
    speaker_emoji = "🧔" if is_socrates else "👸"
    speaker_title = f"{speaker_emoji} {speaker_name}"
    
    # Truncate long messages for display
    display_text = message[:100] + "..." if len(message) > 100 else message
    
    return {
        "speaker": speaker_title,
        "speaker_name": speaker_name,
        "text": display_text,
        "full_text": message,
        "audio": audio_tuple,
        "is_socrates": is_socrates,
        "timestamp": time.time()
    }


def read_message_with_audio_history(history, audio_history, message, is_socrates=True):
    """
    Add message to both text and audio history with proper character attribution.
    """
    character_name = "Socrates" if is_socrates else "Eris"
    
    # Add to text history with character name
    if is_socrates:
        history.append({"role": "assistant", "content": f"**{character_name}**: {message}"})
    else:
        history.append({"role": "user", "content": f"**{character_name}**: {message}"})

    # Generate audio with character-specific voice
    audio = speak_with_voice(message, is_socrates)
    
    # Add to audio history with full character info
    audio_item = create_audio_history_item(message, is_socrates, audio)
    audio_history.append(audio_item)
    
    yield history, audio_history, audio
    
    # Wait for audio to finish
    audio_duration = estimate_audio_duration(audio)
    time.sleep(audio_duration)


def adversarial_chat_with_audio_history(n_rounds, topic, stop_flag=None):
    """
    Enhanced debate between Socrates and Eris with audio history tracking.
    """
    # Initialize chatbot instances (replace with your actual implementations)
    socrates = TestSocrates()
    eris = TestEris()
    # socrates = Socrates()
    # eris = Eris()
    
    history = []
    audio_history = []

    if stop_flag and stop_flag:
        return

    # Eris starts the heated debate
    eris_opening = f"Hey Socrates! Ready to dive into the topic of '{topic}'? I hope you're prepared because I'm going to challenge every assumption you make!"
    
    for update in read_message_with_audio_history(history, audio_history, eris_opening, is_socrates=False):
        if stop_flag and stop_flag:
            return
        yield update

    # Continue with alternating turns
    for round_num in range(n_rounds-1):
        if stop_flag and stop_flag:
            return

        if round_num % 2 == 0:  # Socrates responds
            # Simulate Socrates' philosophical response
            socrates_response = f"Interesting perspective, Eris. But let me ask you this about {topic}: What do you truly mean when you use that term? Are we perhaps operating under different definitions here? Round {round_num + 1}"
            socrates_response = clean_message(socrates_response)
            
            for update in read_message_with_audio_history(history, audio_history, socrates_response, is_socrates=True):
                if stop_flag and stop_flag:
                    return
                yield update

        else:  # Eris responds
            # Simulate Eris' provocative counter-response  
            eris_counter = f"Oh Socrates, there you go again with your endless questions! Sometimes a spade is just a spade. But fine, let's play your game regarding {topic}... Round {round_num + 1}"
            eris_counter = clean_message(eris_counter)
            
            for update in read_message_with_audio_history(history, audio_history, eris_counter, is_socrates=False):
                if stop_flag and stop_flag:
                    return
                yield update


def create_audio_history_chat(audio_history):
    """
    Create a chat-like interface for audio history with individual audio players.
    Uses proper character names throughout.
    """
    if not audio_history:
        return "<div style='text-align: center; color: #666; padding: 20px; font-style: italic;'>No messages from Socrates or Eris yet. Start a debate to see the conversation unfold!</div>"
    
    html_content = """
    <div style='max-height: 500px; overflow-y: auto; padding: 15px; border: 1px solid #ddd; border-radius: 12px; background: linear-gradient(to bottom, #fafafa, #f0f0f0);'>
    """
    
    for i, item in enumerate(audio_history):
        speaker_name = item["speaker_name"]  # "Socrates" or "Eris"
        speaker_emoji = "🧔" if item["is_socrates"] else "👸"
        speaker_color = "#4A90E2" if item["is_socrates"] else "#E24A4A"
        
        # Message alignment - Socrates on right, Eris on left
        alignment = "flex-end" if item["is_socrates"] else "flex-start"
        margin = "margin-left: 50px;" if item["is_socrates"] else "margin-right: 50px;"
        
        # Add message counter
        message_number = i + 1
        
        html_content += f"""
        <div style='display: flex; justify-content: {alignment}; margin-bottom: 20px;'>
            <div style='max-width: 75%; {margin}'>
                <div style='background: white; border-radius: 15px; padding: 15px; box-shadow: 0 3px 8px rgba(0,0,0,0.12); border-left: 5px solid {speaker_color}; position: relative;'>
                    <div style='font-weight: bold; color: {speaker_color}; margin-bottom: 10px; font-size: 15px; display: flex; align-items: center; justify-content: space-between;'>
                        <span>{speaker_emoji} {speaker_name}</span>
                        <span style='font-size: 11px; color: #888; font-weight: normal;'>Message #{message_number}</span>
                    </div>
                    <div style='margin-bottom: 12px; font-size: 14px; color: #333; line-height: 1.5; font-style: italic;'>
                        "{item["text"]}"
                    </div>
                    <div style='margin-top: 10px; padding: 8px; background: #f8f9fa; border-radius: 8px; border: 1px solid #e9ecef;'>
                        <div style='font-size: 11px; color: #666; margin-bottom: 5px;'>🎵 {speaker_name}'s Voice:</div>
                        <div id="audio-player-{i}" style='margin-top: 5px;'>
                            <!-- Audio player will be inserted here -->
                        </div>
                    </div>
                </div>
            </div>
        </div>
        """
    
    html_content += """
    </div>
    <div style='text-align: center; margin-top: 10px; font-size: 12px; color: #666;'>
        💡 Click any audio player above to replay that part of the Socrates vs Eris debate!
    </div>
    """
    return html_content


# Gradio Interface
with gr.Blocks(css="""
    .audio-history {
        border: 2px solid #ddd;
        border-radius: 12px;
        background-color: #f9f9f9;
    }
    .current-audio {
        border: 3px solid #4A90E2;
        border-radius: 12px;
        background-color: #f0f8ff;
        padding: 10px;
    }
    .character-info {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 15px;
        border-radius: 10px;
        margin-bottom: 20px;
    }
    """) as demo:
    
    with gr.Row():
        gr.Markdown("""
        <div class="character-info">
        <h2 style="margin:0; text-align:center;">⚔️ Socrates vs. Eris - The Ultimate Philosophical Battle</h2>
        <p style="margin:10px 0 0 0; text-align:center;">
        Watch as <strong>🧔 Socrates</strong>, the master of questioning, faces off against <strong>👸 Eris</strong>, the goddess of discord and contradiction!
        </p>
        </div>
        """)

    gr.Markdown("""
    **How it works:**
    - **Socrates** uses his deep, contemplative voice to ask probing questions and seek truth
    - **Eris** responds with her dynamic, playful voice full of wit and contradiction  
    - **New Feature**: Browse the complete audio history and replay any message from either character!
    """)

    with gr.Row():
        rounds = gr.Number(label="Number of Rounds", value=4, precision=0, minimum=1, maximum=20)
        topic_input = gr.Textbox(
            label="Debate Topic", 
            placeholder="e.g., What is the meaning of justice? | Is technology making us more or less human? | What defines true friendship?",
            lines=2
        )
        start_btn = gr.Button("🎭 Start the Socrates vs Eris Debate!", interactive=False, variant="primary")

    with gr.Row():
        with gr.Column(scale=1):
            chatbox = gr.Chatbot(
                label="💬 Socrates vs Eris Debate Transcript", 
                type="messages", 
                height=350,
                show_copy_button=True
            )
            current_audio = gr.Audio(
                label="🎵", 
                autoplay=True, 
                elem_classes=["current-audio"]
            )
        
        with gr.Column(scale=1):
            audio_history_chat = gr.HTML(
                label="🎧 Complete Audio History - Socrates & Eris", 
                elem_classes=["audio-history"]
            )
            
            # Pre-create audio players for individual messages
            audio_players = []
            for i in range(20):  # Support up to 20 messages
                audio_player = gr.Audio(
                    visible=False, 
                    label=f"Message {i+1}", 
                    interactive=False,
                    show_download_button=True
                )
                audio_players.append(audio_player)

    # State management
    audio_history_state = gr.State([])

    def reset_interface():
        """Reset all interface elements for a fresh debate"""
        empty_players = [gr.update(visible=False, value=None) for _ in range(20)]
        welcome_msg = "<div style='text-align: center; color: #666; padding: 30px; font-style: italic;'>🎭 Ready for an epic battle of minds between Socrates and Eris!<br/>Choose your topic and hit start!</div>"
        return [[], None, [], welcome_msg] + empty_players

    def update_audio_chat_interface(audio_history):
        """Update the chat interface with proper character names and audio players"""
        html_display = create_audio_history_chat(audio_history)
        
        # Update individual audio players
        audio_updates = []
        for i in range(20):
            if i < len(audio_history):
                item = audio_history[i]
                character = item["speaker_name"]
                label = f"🎵 {character} - Message #{i+1}"
                audio_updates.append(gr.update(
                    visible=True, 
                    value=item["audio"],
                    label=label
                ))
            else:
                audio_updates.append(gr.update(visible=False, value=None))
        
        return [html_display] + audio_updates

    def validate_inputs(topic, n_rounds):
        """Validate that we have proper inputs before starting"""
        valid_topic = bool(topic.strip()) and len(topic.strip()) > 3
        valid_rounds = isinstance(n_rounds, (int, float)) and 1 <= n_rounds <= 20
        return gr.update(interactive=valid_topic and valid_rounds)

    # Event handlers
    reset_outputs = [chatbox, current_audio, audio_history_state, audio_history_chat] + audio_players
    
    # Reset interface when starting new debate
    start_btn.click(fn=reset_interface, outputs=reset_outputs)
    
    # Main debate function with Socrates vs Eris
    start_btn.click(
        fn=adversarial_chat_with_audio_history,
        inputs=[rounds, topic_input],
        outputs=[chatbox, audio_history_state, current_audio]
    )
    
    # Update audio history display when new messages are added
    chat_outputs = [audio_history_chat] + audio_players
    audio_history_state.change(
        fn=update_audio_chat_interface,
        inputs=[audio_history_state],
        outputs=chat_outputs
    )

    # Input validation for better UX
    topic_input.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)
    rounds.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)

    # Add helpful examples
    gr.Markdown("""
    ### 💡 Suggested Debate Topics:
    - **Philosophy**: "What is the nature of reality?" | "Is free will an illusion?"
    - **Ethics**: "When is lying justified?" | "What do we owe to future generations?"  
    - **Society**: "Is technology making us more connected or isolated?" | "What makes a life worth living?"
    - **Science**: "Should we fear artificial intelligence?" | "Is consciousness unique to humans?"
    """)

# if __name__ == "__main__":
# demo.launch(share=True, debug=True)
demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7892
* Running on public URL: https://097d14c49328bd45e1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\gradio\processing_utils.py:753: UserWarning: Trying to convert audio automatically from float64 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7892 <> https://097d14c49328bd45e1.gradio.live
